In [1]:
!pip install -U git+https://github.com/faustomorales/keras-ocr.git
!pip install imgaug
!pip install -U opencv-python
%tensorflow_version 2.x

!gdown --id 12WOvX4IDbmJUTDOdWbV3wWAc4lr5GGmO
!gdown --id 1xCeveu2CyYLNyTeXfsRRTABPwGRYpI82

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/faustomorales/keras-ocr.git to /tmp/pip-req-build-g8t75ruh
  Running command git clone -q https://github.com/faustomorales/keras-ocr.git /tmp/pip-req-build-g8t75ruh
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 930 kB 12.5 MB/s 
     |████████████████████████████████| 9.5 MB 53.1 MB/s 
     |████████████████████████████████| 603 kB 63.3 MB/s 
     |████████████████████████████████| 50 kB 7.5 MB/s 
  Created wheel for keras-ocr: filename=keras_ocr-0.9.0.post2+40bb9bd-py3-none-any.whl size=42441 sha256=395ed38f34cb967c37e2cf7ce5f322a5231d7070ae68addf74cc2ca4a5851340
  Stored in directory: /tmp/pip-ephem-wheel-cache-ifuhp4pf/wheels/20/d0/06/ed290995144ec8dde71d3a91d0d590145445c01be60ea70c9b
  Created wheel for validators: filename=validators-0.1

In [2]:
import random
import string
import math
import itertools
import os
import zipfile
import typing

import numpy as np

import imgaug
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn.model_selection

import keras_ocr

dir = "."

In [3]:
def extract_train_dataset(main_dir="."):
  if not os.path.isdir(main_dir):
    os.mkdir(main_dir)
  train_image_dir = os.path.join(main_dir, "train_images")
  if not os.path.isdir(train_image_dir):
    os.mkdir(train_image_dir)
  training_zip_images_path = os.path.join(".", "Challenge2_Training_Task3_Images_GT.zip")
  if len(os.listdir(train_image_dir)) != 849:
    with zipfile.ZipFile(training_zip_images_path, 'r') as zip_ref:
      zip_ref.extractall(train_image_dir)

In [4]:
def extract_test_dataset(main_dir="."):
  if not os.path.isdir(main_dir):
    os.mkdir(main_dir)
  test_image_dir = os.path.join(main_dir, "test_images")
  if not os.path.isdir(test_image_dir):
    os.mkdir(test_image_dir)
  training_zip_images_path = os.path.join(".", "Challenge2_Test_Task3_Images_GT.zip")
  if len(os.listdir(test_image_dir)) != 1095:
    with zipfile.ZipFile(training_zip_images_path, 'r') as zip_ref:
      zip_ref.extractall(test_image_dir)

In [5]:
def preprocess_train_dataset(dir="."):
  main_dir = os.path.join(dir, "icdar2013")
  extract_train_dataset(main_dir)
  dataset = []

  train_image_dir = os.path.join(main_dir, "train_images")
  train_gt_path = os.path.join(train_image_dir, "gt.txt")

  dataset = []

  with open(train_gt_path, "r", encoding="utf8") as f:
    for line in f:
      image_id, text = line.split(".png,")
      text = text.strip("\"\n ").lower()
      image_path = os.path.join(train_image_dir, image_id + ".png")
      dataset.append((image_path, None, text))
      
  return dataset

In [6]:
def preprocess_test_dataset(dir="."):
  main_dir = os.path.join(dir, "icdar2013")
  extract_test_dataset(main_dir)
  dataset = []

  test_image_dir = os.path.join(main_dir, "test_images")
  test_gt_path = os.path.join(test_image_dir, "gt.txt")

  dataset = []

  with open(test_gt_path, "r", encoding="utf8") as f:
    for line in f:
      image_id, text = line.split(".png,")
      text = text.strip("\"\n ").lower()
      image_path = os.path.join(test_image_dir, image_id + ".png")
      dataset.append((image_path, None, text))
      
  return dataset

In [9]:
main_dir = os.path.join(dir, "icdar2013")
test_image_dir = os.path.join(main_dir, "test_images")
for filename in os.listdir(test_image_dir):
  if filename == "gt.txt":
    print(filename)

gt.txt


In [17]:
train_dataset = preprocess_train_dataset(dir)
test_dataset = preprocess_test_dataset(dir)
train_labels = [(filepath, box, word.lower()) for filepath, box, word in train_dataset]
test = [(filepath, box, word.lower()) for filepath, box, word in test_dataset]

train_labels_extend, test = sklearn.model_selection.train_test_split(test, test_size=0.5, random_state=42)
validation_labels, test_labels = sklearn.model_selection.train_test_split(test, test_size=0.5, random_state=42)

train_labels.extend(train_labels_extend)

In [18]:
print(len(train_labels))
print(len(validation_labels))
print(len(test_labels))

1395
274
274


In [19]:
alphabet = string.digits + string.ascii_letters + '!?., '
recognizer_alphabet = ''.join(sorted(set(alphabet.lower())))

In [20]:
print(recognizer_alphabet)

 !,.0123456789?abcdefghijklmnopqrstuvwxyz


In [27]:
BUILD_PARAMS = {
    "height": 31,
    "width": 200,
    "color": False,
    "filters": (64, 128, 256, 256, 512, 512, 512),
    "rnn_units": (128, 128),
    "dropout": 0.2,
    "rnn_steps_to_discard": 2,
    "pool_size": 2,
    "stn": True,
}

In [28]:
detector = keras_ocr.detection.Detector(weights='clovaai_general')
recognizer = keras_ocr.recognition.Recognizer(
    alphabet=recognizer_alphabet,
    weights='kurapan',
    build_params=BUILD_PARAMS
)
recognizer.compile(metrics=["acc"])
for layer in recognizer.backbone.layers:
    layer.trainable = False

Looking for /root/.keras-ocr/craft_mlt_25k.h5
Provided alphabet does not match pretrained alphabet. Using backbone weights only.
Looking for /root/.keras-ocr/crnn_kurapan_notop.h5


In [29]:
batch_size = 8
augmenter = imgaug.augmenters.Sequential([
    imgaug.augmenters.GammaContrast(gamma=(0.25, 3.0)),
])

(training_image_gen, training_steps), (validation_image_gen, validation_steps) = [
    (
        keras_ocr.datasets.get_recognizer_image_generator(
            labels=labels,
            height=recognizer.model.input_shape[1],
            width=recognizer.model.input_shape[2],
            alphabet=recognizer.alphabet,
            augmenter=augmenter
        ),
        len(labels) // batch_size
    ) for labels, augmenter in [(train_labels, augmenter), (validation_labels, None)]     
]
training_gen, validation_gen = [
    recognizer.get_batch_generator(
        image_generator=image_generator,
        batch_size=batch_size
    )
    for image_generator in [training_image_gen, validation_image_gen]
]

In [30]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=False),
    tf.keras.callbacks.ModelCheckpoint('recognizer_icdar2013.h5', monitor='val_loss', save_best_only=True),
]
recognizer.training_model.fit(
    x=training_gen,
    steps_per_epoch=training_steps,
    validation_steps=validation_steps,
    validation_data=validation_gen,
    callbacks=callbacks,
    epochs=50,
    verbose=1
)

67 / 1395 instances have illegal characters.
Epoch 1/50
174/174 [==============================] - 27s 110ms/step - loss: 13.2725 - acc: 0.0057 - val_loss: 5.0784 - val_acc: 0.0809
Epoch 2/50
174/174 [==============================] - 17s 98ms/step - loss: 3.4950 - acc: 0.2773 - val_loss: 3.6560 - val_acc: 0.3676
Epoch 3/50
174/174 [==============================] - 17s 99ms/step - loss: 2.5933 - acc: 0.4799 - val_loss: 4.3938 - val_acc: 0.3676
Epoch 4/50
174/174 [==============================] - 17s 99ms/step - loss: 1.9046 - acc: 0.5963 - val_loss: 3.2881 - val_acc: 0.3824
Epoch 5/50
174/174 [==============================] - 17s 98ms/step - loss: 1.4031 - acc: 0.6767 - val_loss: 3.1797 - val_acc: 0.4816
Epoch 6/50
174/174 [==============================] - 17s 97ms/step - loss: 1.1273 - acc: 0.7170 - val_loss: 4.1948 - val_acc: 0.3676
Epoch 7/50
174/174 [==============================] - 17s 99ms/step - loss: 0.9698 - acc: 0.7313 - val_loss: 2.9764 - val_acc: 0.5699
Epoch 8/50
174/

In [ ]:
from google.colab import files
files.download('/content/recognizer_icdar2013.h5') 